In [12]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from statapp.common.utils import array_to_multi_indexed_series, survival_function
from statapp.common.preprocessing import load_data
from statapp.attention_analysis.data_acquisition import *
from statapp.attention_analysis.entropy import *
from statapp.attention_analysis.plotting import *
data_path = "../data/attention_data/"

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import tnrange, tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_file_names = {
    "en_ewt" : "English",
    "fr_gsd" : "French",
    "de_gsd" : "German",
    "tr_imst" : "Turkish",
    "ru_gsd" : "Russian",
    "ar_padt" : "Arabic",
    "ug_udt" : "Uyghur",
    "mt_mudt" : "Maltese",
    "id_gsd" : "Indonesian",
    "zh_gsd" : "Chinese"
                  }

In [3]:
tokenizer, model = get_tokenizer_and_model("xlm-roberta-base")

In [44]:
#Constitution des jeux de données
x = np.array([])
y = np.array([])

batch_size = 32

for file_name, language in data_file_names.items():
    
    text = "\n".join(load_data(data_path + file_name + "-ud-test-sent_segmented.txt", sample=1, split_on="\n"))

    tokens = tokenizer.encode(text)
    #print(len(tokens))
    
    for i in range(0,batch_size*(len(tokens)//batch_size),64*batch_size):
        
        att = get_attentions(tokens[i:i+64*batch_size], model, seq_length=64, batch_size=batch_size, as_array=True, verbose=False)
        #remove mean over axis=-2 et flatten over last axis to keep 144 dimensions
        entropies = compute_entropy(att).mean(axis=-1).mean(axis=-1)
        x = np.concatenate((x,entropies)) if x.size else entropies
        
        y = np.concatenate((y,np.array([language]).repeat(batch_size)))
                            
    print("Terminé pour le "+str(language))
    

Token indices sequence length is longer than the specified maximum sequence length for this model (31698 > 512). Running this sequence through the model will result in indexing errors


(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)
(32, 12)


KeyboardInterrupt: 

In [ ]:
#sauvegarde des attentions
import pickle

attention_entropy_data = {"attentions":x, "languages":y}

pickle_out = open("attention_entropy_data","wb")
pickle.dump(attention_entropy_data, pickle_out)
pickle_out.close()

In [43]:
#Sélection d'indices pour train et test
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)

for train_index, test_index in sss.split(x, y):
    x_train, x_test = np.array(x)[train_index], np.array(x)[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]

ValueError: The test_size = 2 should be greater or equal to the number of classes = 10

In [23]:
reg = LogisticRegression().fit(x_train, y_train)
reg.score(x_test, y_test)

C:\Users\Eric\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Found array with dim 3. Estimator expected <= 2.